In [1]:
# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split,RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, confusion_matrix
from pycaret.classification import *
exec(open('NuestrasFunciones.py').read())
import statsmodels.formula.api as smf
from statsmodels.formula.api import logit 
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Carga datos depurados

In [3]:
train = pd.read_hdf('DatosFinal_Train.h5', 'df')
test = pd.read_hdf('DatosFinal_Test.h5', 'df')

train.rename(columns={'Mayor65_1.0': 'Mayor65_1'}, inplace=True)
test.rename(columns={'Mayor65_1.0': 'Mayor65_1'}, inplace=True)

# Eliminar la columna 'ID'
train.drop(columns=['ID'], inplace=True)
test_id = test['ID']
test.drop(columns=['ID'], inplace=True)

train.info()
test.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6343 entries, 0 to 6352
Data columns (total 59 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Fuga                                    6343 non-null   uint8  
 1   month                                   6343 non-null   float32
 2   FacturaMes                              6343 non-null   float32
 3   FacturaTotal                            6343 non-null   float32
 4   prop_missings                           6343 non-null   uint8  
 5   Genero_Male                             6343 non-null   uint8  
 6   Mayor65_1                               6343 non-null   uint8  
 7   Conyuge_Yes                             6343 non-null   uint8  
 8   PersCargo_Yes                           6343 non-null   uint8  
 9   Telf_serv_Yes                           6343 non-null   uint8  
 10  VariasLineas_Yes                        6343 non-null   uint8  
 

### Regresión Logística


In [4]:
X_train = train.drop(columns=['Fuga'])
y_train = train['Fuga']



In [5]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Comprobamos la nueva distribución de clases
print(y_train_res.value_counts())



Fuga
0    4657
1    4657
Name: count, dtype: int64


In [6]:
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_res, y_train_res, test_size=0.2, random_state=42)


In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_split)
X_val_scaled = scaler.transform(X_val_split)
X_test_scaled = scaler.transform(test)



In [8]:
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train_split)


LogisticRegression(random_state=42)

In [9]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train_split)



LogisticRegression(random_state=42)

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_val_pred = model.predict(X_val_scaled)
print("Accuracy:", accuracy_score(y_val_split, y_val_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val_split, y_val_pred))
print("Classification Report:\n", classification_report(y_val_split, y_val_pred))


Accuracy: 0.8604401502952228
Confusion Matrix:
 [[807 107]
 [153 796]]
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.88      0.86       914
           1       0.88      0.84      0.86       949

    accuracy                           0.86      1863
   macro avg       0.86      0.86      0.86      1863
weighted avg       0.86      0.86      0.86      1863



In [11]:
test_predictions = model.predict(X_test_scaled)

# Crear un DataFrame con las predicciones y el ID
test_results = pd.DataFrame({'ID': test_id, 'Predicted_Fuga': test_predictions})

# Guardar las predicciones en un archivo
test_results.to_csv('test_predictions.csv', index=False)